# Moview poster to category 

I wanted to explore the idea of classifing the category of the movie (romance, action, etc) based on the poster while also exploring [fast.ai](https://www.fast.ai/).
I used [this kaggle dataset](https://www.kaggle.com/neha1703/movie-genre-from-its-poster) which were obtained from [IMDB](https://www.imdb.com/).

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
from fastai.vision import *
import download

np.random.seed(42)

## Pandas

Let's start off by exploring the dataset using pandas, we need to clean some some of movies that have no genre

In [ ]:
path = Path(os.path.expanduser('~'))/'data/kaggle_posters'
df = pd.read_csv(
    path/'MovieGenre.csv', encoding="ISO-8859-1", 
    dtype={'imdbId': str, 'Genre': str, 'Poster': str})

In [ ]:
df = df.drop(columns=['Imdb Link', 'Title', 'IMDB Score'])
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df = df[~df['Genre'].isnull()]

In [ ]:
if not (path/'posters').ls():
    await download.download_data({
            x['Poster']: path/'posters/{}'.format(x['imdbId']) for _, x in df.iterrows()})

In [ ]:
verify_images(path/'posters', delete=True)

In [ ]:
df = df[df['imdbId'].isin([x.stem for x in (path/'posters').ls()])]
df.shape

Check how many movies per-genre we have and remove the ones that have less than 1K examples

In [ ]:
categories = set([g for s in df['Genre'].tolist() for g in s.split('|')])
l = []
for c in categories:
    l.append((c, df[df['Genre'].str.contains(c)].shape[0]))
    
['{} ({})'.format(*x) for c in sorted(l, key=lambda x: x[1])]

In [ ]:
unwanted = [x[0] for x in l if x[1] < 1000]
cleaned = df
for u in unwanted:
    cleaned = cleaned[~cleaned['Genre'].str.contains(u)]

## Create the Databuch

In [ ]:
src = (ImageList.from_df(cleaned, path/'posters', suffix='.jpg')
               .split_by_rand_pct()
               .label_from_df(cols=1, label_delim='|'))
src

I didn't use transformations because the posters will allways appear the same way in real life 

In [ ]:
data = (src.transform([[], []], size=64)
               .databunch(bs=128).normalize(imagenet_stats))
print(data.classes)
data.show_batch(rows=3, figsize=(7,8))

## Training

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[accuracy_thresh, fbeta])

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 1e-2
learn.fit_one_cycle(6, slice(lr))
learn.save('stage-1')

In [ ]:
learn.recorder.plot_losses()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_multi_top_losses(figsize=(2,2))

In [ ]:
# helper to get the worst k classes according to fbeta score. It returns prec and rec for all of them
def worst_classes(self, k=5, thresh:float=0.2, beta:float=2, eps:float=1e-9, sigmoid:bool=True):
    # this is basically the code for fbeta
    beta2 = beta ** 2
    y_pred = (self.probs>thresh).float()
    
    if sigmoid: y_pred = y_pred.sigmoid()    
    
    y_true = self.y_true.float()
    TP = (y_pred*y_true).sum(dim=0)
    prec = TP/(y_pred.sum(dim=0)+eps)
    rec = TP/(y_true.sum(dim=0)+eps)
    fbeta = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    
    sorted_res, sorted_idx = fbeta.sort()
    total_true = y_true.sum(0)
    return [(self.data.classes[i], total_true[i] fbeta[i], prec[i], rec[i]) for i in sorted_idx[:k]]

In [ ]:
worst_classes(interp, k=data.classes.c)

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, slice(3e-5, lr/5))
learn.save('stage-2')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
worst_classes(interp, k=data.c)

Use bigger images

In [ ]:
data = (src.transform([[], []], size=128)
               .databunch(bs=128).normalize(imagenet_stats))
data.show_batch(rows=3, figsize=(7,8))
learn.data=data

In [ ]:
learn.freeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 1e-3
learn.fit_one_cycle(4, slice(lr))
learn.save('stage-1-128')

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, slice(1e-5, lr/5))
learn.save('stage-2-128')

126 image size

In [ ]:
data = (src.transform([[], []], size=256)
               .databunch(bs=64).normalize(imagenet_stats))
learn.data=data

In [ ]:
learn.freeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 1e-3
learn.fit_one_cycle(4, slice(lr))
learn.save('stage-1-256')

In [ ]:
learn.fit_one_cycle(2, slice(1e-5, lr/5))
learn.save('stage-2-256')

In [ ]:
learn.recorder.plot_losses()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
t2 = worst_classes(interp, k=data.c, thresh=0.2)
t5 = worst_classes(interp, k=data.c, thresh=0.5)
t9 = worst_classes(interp, k=data.c, thresh=0.9)
for i in range(len(t2)):
    print('{}: {} | {} | {}'.format(t2[i][0], t2[i][1], t5[i][1], t9[i][1]))

In [ ]:
worst_classes(interp, k=data.c, thresh=0.2)

In [ ]:
loader.export()